In [2]:
!pip install razdel

## Импорты

In [3]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 
from scipy.sparse import lil_matrix
from collections import Counter
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')  
nltk.download('punkt')
nltk.download('punkt_tab')

In [5]:
news = open('lenta.txt', encoding="utf-8").read()

In [6]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [7]:
norm_news = normalize(news)

In [8]:
vocab_news = Counter(norm_news)

In [ ]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

In [14]:
sentences_news = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news[:5000000], language = "russian")]

In [44]:
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [45]:
unigrams_news = Counter()
bigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence))

In [ ]:
unigrams_news

In [47]:
bigrams_news

Counter({'<start> об этом': 657,
         '<start> по словам': 626,
         'со ссылкой на': 606,
         '<start> как сообщает': 600,
         'сообщает риа новости': 500,
         '<start> кроме того': 452,
         'в связи с': 417,
         'риа новости <end>': 361,
         'по его словам': 355,
         '<start> по его': 336,
         '<start> напомним что': 331,
         'о том что': 327,
         'в настоящее время': 312,
         '<start> по данным': 310,
         '<start> по мнению': 293,
         'в том числе': 248,
         '<start> при этом': 224,
         'риа новости со': 203,
         'как сообщает риа': 198,
         'новости со ссылкой': 198,
         '<start> как сообщили': 181,
         'в соответствии с': 175,
         '<start> в настоящее': 172,
         'риа новости в': 169,
         'в то же': 168,
         'то же время': 168,
         '<start> как сообщил': 163,
         '<start> в то': 157,
         'на северном кавказе': 156,
         'в ближайшее время': 1

In [50]:

def compute_joint_proba_markov_assumption(phrase, word_counts, bigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
        word1, word2, word3 = ngram.split()
        if word1 in word_counts and ngram in bigram_counts:
            prob += np.log(bigram_counts[ngram]/word_counts[word1])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

In [51]:
compute_joint_proba_markov_assumption('красивый большой дом', unigrams_news, bigrams_news)

np.float64(7.999999999999997e-15)

In [52]:
matrix_news = lil_matrix((len(unigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}



for ngram in bigrams_news:
    word1, word2, word3 = ngram.split()
    matrix_news[word2id_news[word1], word2id_news[word2]] =  (bigrams_news[ngram]/
                                                                     unigrams_news[word1])

ValueError: too many values to unpack (expected 2)

In [ ]:
matrix_news

<List of Lists sparse matrix of dtype 'float64'
	with 380448 stored elements and shape (71987, 71987)>
  Coords	Values
  (0, 1)	0.00015691689681144864
  (0, 2)	0.00354632186793874
  (0, 4)	0.004268139593271403
  (0, 9)	3.138337936228973e-05
  (0, 10)	3.138337936228973e-05
  (0, 12)	0.006935726839066031
  (0, 14)	0.004770273663068039
  (0, 20)	0.10158799899573186
  (0, 24)	9.41501380868692e-05
  (0, 25)	3.138337936228973e-05
  (0, 29)	0.0002196836555360281
  (0, 31)	0.00025106703489831785
  (0, 35)	6.276675872457946e-05
  (0, 39)	6.276675872457946e-05
  (0, 43)	0.005962842078835049
  (0, 46)	0.00025106703489831785
  (0, 47)	0.00012553351744915893
  (0, 49)	0.08024730102937484
  (0, 50)	3.138337936228973e-05
  (0, 51)	0.0011298016570424304
  (0, 62)	3.138337936228973e-05
  (0, 63)	3.138337936228973e-05
  (0, 65)	3.138337936228973e-05
  (0, 67)	0.009634697464222948
  (0, 74)	0.0010984182776801407
  :	:
  (71962, 14)	1.0
  (71963, 71964)	1.0
  (71964, 33972)	1.0
  (71965, 2)	1.0
  (71966, 

In [29]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
        # просто выбирать наиболее вероятное продолжение не получится
        # можете попробовать раскоментировать следующую строчку и посмотреть что получается
#         chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [31]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

вторая по вопросам 
 мэор собирается штурмовать грозный и медперсонала 
 по решению проблемы нагорного карабаха 
 листовки долетели далеко не так как сообщает associated press 102 человека не видел ничего катастрофического не верят в палау центральная избирательная комиссия шамгара расследовавшей историю о крушении 
 министр иностранных дел ливана заблокировало импорт за границу двух третей американских инвестиций примерно 10 самых кровавых в ситуации среди сил направил дело будет в настоящий момент осуществляют перегруппировку сил на черно-белых фотографиях были убиты около трети американцев расходовать свои дома не следует реклама мыла выросло до сих пор было никаких опасных преступников 
 сделка 29
